## Classification Accuracy

In [ ]:
# read the data into pandas Dataframe
import pandas as pd

col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']

pima = pd.read_csv('pima-indians-diabetes.data', header=None, names=col_names)
pima.head()

### predicting diabetes status of a patient given health conditions

In [ ]:
# define X and y
feature_cols = ['pregnant', 'insulin', 'bmi', 'age']
X = pima[feature_cols]
y = pima.label

In [ ]:
# split X and y in train and tgest set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg.fit(X_train, y_train)

In [ ]:
# prediction for test set
y_pred = logReg.predict(X_test)

In [ ]:
# classification accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

### Null accuracy: accuracy that can be achieved by always predicting most frequent class

In [ ]:
# examining class distribution of testing set
y_test.value_counts()

In [ ]:
# calculate percentage of ones
y_test.mean()

In [ ]:
# calculate percentage of zeros
1 - y_test.mean()

In [ ]:
# calculating null accuracy( for binary classification coded as 0/1)
max(y_test.mean(), 1 - y_test.mean())

In [ ]:
# calculating null accuracy for multi-class classification
y_test.value_counts().head(1) / len(y_test)

### comparing true and predicted response values

In [ ]:
# print first 25 true and predicted responses
print('True:', y_test.values[0:25])
print('Pred:', y_pred[0:25])

## Confusion matrix

In [ ]:
# first argument is true values, second argument is predicted values
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
confusion = metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

### Metrics computed from a confusion matrix

### Classification Accuracy: Overall, how often is the classifier correct ?

In [ ]:
print((TP+TN)/ float(TP+TN+FP+FN))
print(metrics.accuracy_score(y_test, y_pred))

### Classification Error: Overall, how often is the classifier Incorrect ?

In [ ]:
print((FP+FN)/ float(TP+TN+FP+FN))
print(1-metrics.accuracy_score(y_test, y_pred))

### Sensitivity: When Actual value is positive, how often is the prediction correct ?

- How "sensitive" is the classifier to detecting positive instance
- ALso known as "True Positive Rate" or "Recall"

In [ ]:
print(TP/ float(TP+FN))
print(metrics.recall_score(y_test, y_pred))

### Specificity: When actual value is negative, how often is the prediction correct ?

- How "specific" (or "selective") is the classifier in predicting positive instances ?

In [ ]:
print(TN/ float(TN+FP))

### False Positive Rate: When actual value is negative, how often is the prediction incorrect ?

In [ ]:
print(FP / float(TN+FP))

# or 1 - specificity
print(1 - (TN/ float(TN+FP)))

### Precision: When Positive value is predicted, how often is the prediction correct ?

In [ ]:
print(TP / float(FP+TP))
print(metrics.precision_score(y_test, y_pred))

### Adjusting the classification threshold

In [ ]:
# print the first 10 predicted responses
logReg.predict(X_test)[0:10]

In [ ]:
# print the first 10 predicted probabilities of class
logReg.predict_proba(X_test)[0:10, :]

In [ ]:
# storing predicted prob for class 1
y_pred_prob = logReg.predict_proba(X_test)[:, 1]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.hist(y_pred_prob, bins=8)
plt.xlim(0, 1)
plt.title('Histogram of predicted probabilities')
plt.xlabel('Predicted probability of diabetes')
plt.ylabel('Frequency')

### Decreaing the classification threshold for predicting diabetes in order to increase the sensitivity

In [ ]:
# predicts diabetes if the predicted probability is gretaer than 0.3
from sklearn.preprocessing import binarize
y_pred = binarize([y_pred_prob], 0.3)[0]

In [ ]:
# print first 10 predicted probabilities
y_pred_prob[0:10]

In [ ]:
# print first 10 predicted classes with lower threshold
y_pred[0:10]

In [ ]:
# previous confusion matrix(with threshold 0.5)
print(confusion)

In [ ]:
# confusion metrics with threshold 0.3
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
# checking sensitivity
print(metrics.recall_score(y_test, y_pred))

In [ ]:
# checking specificity
print(80/ float(80+50))

### ROC curves and Area under the curve (AUC)

- To check various threshold for sensitivity and specificity

In [ ]:
# first argument is true values, second argument is predicted probabilities
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for diabetes classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

In [ ]:
# define function that accepts threshold and prints sensitivity and specificity
def evaluate_threshold(threshold):
    print('Sensitivity:', tpr[thresholds > threshold][-1])
    print('Specificity:', 1 - fpr[thresholds > threshold][-1])

In [ ]:
evaluate_threshold(0.5)

In [ ]:
evaluate_threshold(0.3)

In [ ]:
# AUC is the percenatage of the ROC plot that is underneath the curve
print(metrics.roc_auc_score(y_test, y_pred_prob))